In [1]:
#include "../common.hpp"

using namespace std;
using namespace std::string_literals;

## RValue References

- [_rvalue_](http://en.cppreference.com/w/cpp/language/value_category) (right hand value) is an unnamed temporary
- `T&&` is used to denote an [_rvalue reference_](http://en.cppreference.com/w/cpp/language/reference) a reference that can only bind to a temporary

```cpp
string str = "Hello"s;
string&& ref = str;
```
---
```cpp
input_line_10:3:10: error: rvalue reference to type 'basic_string<...>' cannot bind to lvalue of type 'basic_string<...>'
string&& ref = str;
         ^     ~~~
```

In [2]:
string&& ref = "Hello"s;

- A temporary value is safe to _consume_
- Useful to avoid copies
- A constructor taking the class type by rvalue reference is known as a _move constructor_
    - Similar to a copy constructor but it consumes it's argument

In [3]:
class movable {
    int* _some_data;
public:
    movable(movable&& x) noexcept : _some_data{x._some_data} // consume x
    { x._some_data = nullptr; } // leave x destructible
    
    //...
};

### Return Value Optimization
- _Return value optimization_ (RVO) or [_copy elision_](http://en.cppreference.com/w/cpp/language/copy_elision) avoids a copy (or move) on return by constructing the result in place
- RVO applies to _local named values_ and rvalue results
- Allowed optimziation since C++03, required by C++17

In [4]:
annotate f() {
    annotate x;
    return x;
}

annotate y = f();

annotate ctor


- Arguments to functions are in the caller scope
- RVO applies to passing an argument by value

In [5]:
void g(annotate x) { }
g(f());

annotate ctor
annotate dtor


- RVO does not apply to returning value argument
- C++11 defines returning a value argument as a _move_ operation

In [6]:
annotate h(annotate x) {
    return x;
}

annotate z = h(f());

annotate ctor
annotate move-ctor
annotate dtor


### Using RValue Refs and RVO to Avoid Copies
#### Make Classes Movable
- Provide a move constructor and move assignment operator
    - Compiler will provide them implicitely if
        - there are no user declared copy constructors, copy assignment operators, or destructors
        - all non-static data members and base classes are movable
    - To ensure you have them, declare them `= default`
    - Move constructor and move assignment should be declared `noexcept`
    - Post-condition of moved from object is _partially formed_ & can alias rhs for move assignment
    - Otherwise can assume no aliasing

Example:

In [7]:
class example_01 {
    int* _data;
public:
    explicit example_01(int x) : _data(new int(x)) { }
    ~example_01() { delete _data; }
    
    example_01(const example_01&) = delete;
    example_01& operator=(const example_01&) = delete;
    
    example_01(example_01&& x) noexcept : _data(x._data) { x._data = nullptr; }
    example_01& operator=(example_01&& x) noexcept {
        delete _data;
        _data = x._data;
        x._data = nullptr;
        return *this;
    }
    
    explicit operator int () { return *_data; }
};

In [8]:
class example_02 {
    unique_ptr<int> _data;
public:
    explicit example_02(int x) : _data(make_unique<int>(x)) { }
    // implicit dtor
    
    // implicit deleted copy-ctor and copy-assignment
    
    /*
        move-ctor and move-assignment would be provided by default, but declaring
        them ensures they are provided and correct.
    */
    example_02(example_02&&) noexcept = default;
    example_02& operator=(example_02&&) noexcept = default;
    
    explicit operator int () { return *_data; }
};

#### The Self Swap Problem
What is the post condition of:

In [9]:
string s = "Hello World!";
swap(s, s);

In [10]:
cout << s << endl;

Hello World!


`std::swap()` is defined as:
```cpp
template <class T>
void swap(T& a, T& b) {
    T tmp = move(a);
    a = move(b); // if a and b alias, then b has been moved from
    b = move(tmp);
}
```

```cpp
    example_01& operator=(example_01&& x) noexcept {
        delete _data;
        _data = x._data;
        x._data = nullptr;
        return *this;
    }
```

Is this okay?

In [11]:
example_01 e1(42);
swap(e1, e1);

In [12]:
cout << static_cast<int>(e1) << endl;

42


Class Break - resume here 2018-02-07

#### Use Return Values, Not Out Arguments

- Out-arguments defeat RVO

In [13]:
void output_01(annotate& out) {
    annotate tmp;
    // fill in tmp
    out = tmp;
}

annotate a1;
output_01(a1);

annotate ctor
annotate ctor
annotate assign
annotate dtor


In [14]:
annotate output_02() {
    annotate tmp;
    // fill in tmp
    return tmp;
}

annotate a2 = output_02();

annotate ctor


##### Further Reading

[Stop Using _Out_ Arguments](http://stlab.cc/tips/stop-using-out-arguments.html)

#### Pass _sink_ arguments by value and return, or swap or move into place

- A _sink argument_ is an argument whose value is returned or stored
- Most constructor arguments are sink arguments
- The argument to assignment is a sink argument

In [15]:
string append(string str, const char* suffix) {
    str += suffix;
    return str;
}

In [16]:
annotate append(annotate str) {
    // modify str
    return str;
}

auto str = append(annotate());

annotate ctor
annotate move-ctor
annotate dtor


In [17]:
class example_03 {
    annotate _member;
public:
    explicit example_03(annotate data) : _member(move(data)) { }
};

example_03 e_03{annotate()};

annotate ctor
annotate move-ctor
annotate dtor


`std::move()` is a cast to an rvalue reference.

In [18]:
class example_04 {
    annotate* _member;
public:
    example_04() : _member(new annotate()) { }
    ~example_04() { delete _member; }
    
    example_04(const example_04& x) : _member(new annotate(*x._member)) { }
    example_04(example_04&& x) : _member(x._member) { x._member = nullptr; }
    
    // this assignment handles both move and copy
    example_04& operator=(example_04 x) noexcept {
        delete _member;
        _member = x._member;
        x._member = nullptr;
        return *this; }
};

In [19]:
example_04 e41;
example_04 e42;

annotate ctor
annotate ctor


In [20]:
e41 = e42; // copy

annotate copy-ctor
annotate dtor


@0x7f0f240a2088

In [21]:
e41 = move(e42);

annotate dtor


##### Advantages to by-value assignment

- Single implementation for copy and move assignment
- Transactional (strong exception guarantee) for copy assignment
- Handles self-copy (and usually self-move in moved from case)

##### Disadvantages

- Potential, significant, performance loss on copy assignment
    - [Howard Hinnant - _Everything You Ever Wanted To Know About Move Semantics_](https://www.youtube.com/watch?v=vLinb2fgkHk)

#### Rvalue Member Functions

- `this` is a hidden argument to a member function
- `*this` may be an rvalue

In [22]:
class example_05 {
    annotate _name;
public:
    const annotate& name() const { return _name; }
};

auto name_01 = example_05().name();

annotate ctor
annotate copy-ctor
annotate dtor


In [23]:
class example_06 {
    annotate _name;
public:
    const annotate& name() const& { return _name; }
    annotate name() && { return move(_name); }
};

auto name_02 = example_06().name();

annotate ctor
annotate move-ctor
annotate dtor


#### Forward Declare Argument and Result Type

- You can use pointer or reference to an incomplete type in an interface

- You can also use an incomplete type as a value argument and result type

In [24]:
class example_07; // forward declaration

const example_07& function_01(const example_07&); // You know this works

In [25]:
example_07 function_02(example_07); // This works also!

### Issue

- Assigning an expression to a temporary can impose a performance penalty

In [26]:
annotate g1(annotate x) { return x; }
annotate x;

annotate ctor


In [27]:
auto v1 = g1(f());

annotate ctor
annotate move-ctor
annotate dtor


In [28]:
auto tmp2 = f();
auto v2 = g1(tmp2);

annotate ctor
annotate copy-ctor
annotate move-ctor
annotate dtor


In [29]:
auto tmp3 = f();
auto v3 = g1(move(tmp3));

annotate ctor
annotate move-ctor
annotate move-ctor
annotate dtor


### Summary Recommendations

- Make classes movable
- Use return values, not out arguments
- Pass sink arguments by value and move into place or return
- Forward declare argument and result types

### Homework

- Apply one or more of the recommendations to code in your product
- Measure the results:
    - Runtime performance of an optimized build
    - (and/or) Binary size of an optimized build
    - (and/or) Compile time of a debug build
    - (and/or) Resulting source line count delta (indicator of readability)
- Report the results on the class wiki: [git.corp.adobe.com/better-code/class/wiki](git.corp.adobe.com/better-code/class/wiki)